# SQL 

# Описание проекта

Проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 
Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

---

## Импорт базы данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

---

## Анализ таблиц

In [2]:
pd.io.sql.read_sql('SELECT * FROM books',con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
pd.io.sql.read_sql('SELECT * FROM authors',con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
pd.io.sql.read_sql('SELECT * FROM publishers',con = engine).head(10)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [5]:
pd.io.sql.read_sql('SELECT * FROM ratings',con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
pd.io.sql.read_sql('SELECT * FROM reviews',con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<td bgcolor="#CCCCFF"> 
    
**В таблице publishers есть неочевидные дубликаты типа: Alfred A. Knopf и Alfred A. Knopf Books for Young Readers. Но им присвоен разный ID. Хотя, вероятно, ID присвоен в алфавитном порядке нарастающим числом. Тем не менее, стоит избавиться от таких дубликатов, например, с помощью лемматизации.**
    
</td>

---

## Выполнение запросов

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
pd.io.sql.read_sql(''' SELECT 
                        COUNT(book_id) AS books_count
                       FROM 
                        books 
                       WHERE 
                         CAST(publication_date AS date) > '2000-01-01' '''
                      ,con = engine)

,books_count
0,819


**819 (81%) книг из наших данных вышли после 1 января 2000 года**

In [8]:
pd.io.sql.read_sql(''' SELECT 
                        COUNT(book_id) AS books_count
                       FROM 
                        books'''
                      ,con = engine)

,books_count
0,1000


---

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
pd.io.sql.read_sql(''' SELECT
                         n1.title,
                         n1.mean_rating, 
                         n2.review_count
                       FROM 
                        (SELECT 
                          books.title,
                         AVG(ratings.rating) AS mean_rating
                         FROM 
                          ratings
                          left join books on ratings.book_id = books.book_id
                         GROUP BY
                          books.title) AS n1
                        left join 
                        (SELECT 
                          books.title,
                         COUNT(reviews.review_id) AS review_count
                         FROM 
                          reviews
                         left join books on reviews.book_id = books.book_id
                         GROUP BY
                          books.title) AS n2 on (n1.title = n2.title)
                        '''
                      ,con = engine)

,title,mean_rating,review_count
0,The Count of Monte Cristo,4.217391,5.0
1,Count Zero (Sprawl #2),2.500000,2.0
2,The Botany of Desire: A Plant's-Eye View of th...,3.500000,2.0
3,The Poisonwood Bible,4.363636,5.0
4,The Canterbury Tales,3.333333,3.0
...,...,...,...
994,Of Love and Other Demons,4.500000,2.0
995,In the Heart of the Sea: The Tragedy of the Wh...,3.333333,3.0
996,Welcome to Temptation (Dempseys #1),5.000000,2.0
997,World's End (The Sandman #8),4.500000,2.0


<td bgcolor="#CCCCFF"> 
    
**Проверка:**
    
</td>

In [10]:
#количество уникальных book_id в таблице books
pd.io.sql.read_sql(''' SELECT 
                        COUNT(DISTINCT book_id)
                       FROM 
                        books
                        '''
                      ,con = engine)

,count
0,1000


---

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [11]:
pd.io.sql.read_sql(''' SELECT 
                        publishers.publisher,
                        COUNT(books.book_id) AS books_count
                       FROM 
                        publishers
                        left join books on publishers.publisher_id = books.publisher_id
                       WHERE
                        books.num_pages > 50
                       GROUP BY 
                        publishers.publisher
                       ORDER BY
                        books_count DESC
                       LIMIT 1
                       
                        '''
                      ,con = engine)

,publisher,books_count
0,Penguin Books,42


---

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
pd.io.sql.read_sql(''' SELECT 
                        authors.author,
                        AVG(ratings.rating) AS mean_rating,
                        COUNT(ratings.rating_id) AS ratings_count
                       FROM 
                        ratings
                        left join books on ratings.book_id = books.book_id
                        left join authors on authors.author_id = books.author_id
                       GROUP BY
                        author
                       HAVING
                        COUNT(ratings.rating_id) > 50
                       ORDER BY
                        mean_rating DESC
                       LIMIT 1
                        '''
                      ,con = engine)

,author,mean_rating,ratings_count
0,J.K. Rowling/Mary GrandPré,4.288462,312


---

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
pd.io.sql.read_sql(''' SELECT
                         AVG(n2.count_reviews)
                       FROM 
                        (SELECT
                        ratings.username,
                        COUNT(ratings.rating_id) AS ratings_count
                       FROM 
                        ratings
                       GROUP BY
                        ratings.username
                       HAVING
                        COUNT(ratings.rating_id) > 50) AS n1
                        inner join 
                        (SELECT
                        reviews.username,
                        COUNT(reviews.review_id) AS count_reviews
                       FROM 
                        reviews
                       GROUP BY
                        reviews.username) AS n2 on (n1.username = n2.username)
                        '''
                      ,con = engine)

,avg
0,24.333333


---

**ВЫВОД**
    
    1. 819 книг вышло после 1 января 2000 года (это 81% от всего количества)
    2. Для каждой книги посчитали количество обзоров и среднюю оценку
    3. Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц
    4. J.K. Rowling - автор с самой высокой средней оценкой книг
    5. 24 обзора в среднем делают пользователи, которые поставили больше 50 оценок